# Tutorial: FMR Model Analysis using Jupyter Notebook

## Introduction

In this tutorial, we will walk through the process of Ferromagnetic Resonance (FMR) Model Fitting using Python. The provided code covers data loading, preprocessing, model development, fitting, smoothing, and visualization.

## Prerequisites

Before you begin, ensure you have the following libraries installed:

- NumPy
- Pandas
- Matplotlib
- SciPy
- scikit-image (skimage)
- scikit-learn (sklearn)

You can install these libraries using pip on your Jupyter Notebook if not already installed:

```bash
!pip install numpy 
!pip install pandas 
!pip install matplotlib 
!pip install scipy 
!pip install scikit-image 
!pip install scikit-learn

## List of Libraries
```python
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, convolve
from scipy.interpolate import interp2d
from scipy.signal import spline_filter, medfilt2d, savgol_filter
from skimage import restoration
import warnings
warnings.filterwarnings('ignore')
```

## Data Preparation

## Step 1: Set Up Your Data Directory

```python
# Set your data directory
dir = r'C:\Users\physlab\Desktop\August_data\NiFe_2Mode_R'
```

## Step 2: Read and Preprocess Data
   - Reading data from CSV files in the specified directory.
   - Storing raw data in a panadas dataframe.

```python
def numOfValues():
    for count in range(1000000):
        if os.path.join(dir, os.listdir(dir)[count]).endswith('csv'):
            totalValues = pd.read_csv(os.path.join(dir, os.listdir(dir)[count])).shape[0]
            return totalValues

# Adding a dummy row
channel1 = np.zeros(numOfValues())
channel2 = np.zeros(numOfValues())

for file in os.listdir(dir):
    path = os.path.join(dir, file)
    if path.endswith('.csv'):
        df = pd.read_csv(path)
        channel1 = np.row_stack((channel1, df.iloc[:, 2].values))
        channel2 = np.row_stack((channel2, df.iloc[:, 3].values))

# Removing the dummy row
channel1 = channel1[1:][:] 
channel2 = channel2[1:][:]
```

## Step 3: Load and View Raw Results

* After loading the data, plot the graphs at the first and last frequencies and decide the ideal field range.

```python
field = df.iloc[:, 1].values
signal = channel2[0][:]
plt.figure(figsize=(6,4))
plt.plot(field, signal, 'k')
plt.xlabel('Field (Oe)')
plt.ylabel('dp/dH (a.u.)')
plt.title('First Signal')

signal = channel2[-1][:]
plt.figure(figsize=(6,4))
plt.plot(field, signal, 'k')
plt.xlabel('Field (Oe)')
plt.ylabel('dp/dH (a.u.)')
plt.title('Last Signal')
```

## Step 4: Set Frequency Bounds and Interval

Here, you have to specify lower and upper frequency bounds of the dataset you're using.

The frequency values in my dataset went from 2 GHz to 6 GHz:
```python
freq_lowerbound = 2 # minimum value of frequency sweep
freq_upperbound = 6 # maximum value of frequency sweep
interval = 0.1 # frequency interval
```

## Step 5: FMR Model Development

```python
# FMR Spectrum Fit Equation
_dp_dh_script = '''
def dp_dh_fitfunct(h, %(prefix)sK1, %(prefix)sK2, %(prefix)sH_FMR, %(prefix)sDH):
    K1 = %(prefix)sK1
    K2 = %(prefix)sK2
    H_FMR = %(prefix)sH_FMR
    DH = %(prefix)sDH
    dh = h - H_FMR
    denom = (DH**2 + dh**2)**2
    return (-K1*2*dh*DH - K2*(DH**2-dh**2))/denom
'''

def dp_dh_model(prefix=''):
    expr = 'dp_dh_fitfunct(x, %(prefix)sK1, %(prefix)sK2, %(prefix)sH_FMR, %(prefix)sDH)' % {'prefix': prefix}
    script = _dp_dh_script % {'prefix': prefix}
    return lmfit.models.ExpressionModel(expr, independent_vars=['x'], init_script=script)
```

### Now, your FMR Spectrum Fit Model is ready!


### Set the Field and Frequency Range

Set the optimal range of your data _according to your dataset_.

I chose the frequency range from 2 GHz to 6 GHz and field from 15 to 500 Oe.

```python
# Frequency range that you want to analyze
lower_freq = 2
upper_freq = 6

# Field range that you want to analyze
lower_field = 15
upper_field = 500
```

### This code block fits the data and stores it in a CSV file in the specified directory
```python
fit = np.zeros(len(field_used))

# Create an empty DataFrame to store parameter values and uncertainties
parameter_values_df = pd.DataFrame(columns=['Frequency (GHz)', 'K1', 'K2', 'H_FMR', 'DH', 'Intercept', 'Slope'])

# Loop over frequencies
for i in range(lower_freq, len(freq_used)):
    signal = channel[i][lower_field:upper_field]
    dh = field_used[signal.argmax()] - field_used[signal.argmin()]
    K1 = (signal.max() - signal.min()) / 2 * 36 / 25 * 5 * dh**2
    K2 = 0
    H_fmr = (field_used[signal.argmin()] + field_used[signal.argmax()]) / 2

    # Create the model
    peak = dp_dh_model('pA_')
    bg = lmfit.models.LinearModel(prefix='bg_')
    model = peak + bg

    # Set initial parameters
    pars = model.make_params()
    pars['pA_K1'].set(K1)
    pars['pA_K2'].set(K2)
    pars['pA_H_FMR'].set(H_fmr)
    pars['pA_DH'].set(dh)
    pars['bg_intercept'].set(0)
    pars['bg_slope'].set(0)

    # Fit the model to your data
    fit_result = model.fit(signal, params=pars, x=field_used)

    fitX = np.linspace(field_used.min(), field_used.max(), len(field_used))
    fitY = model.eval(fit_result.params, x=fitX)
    pk1Y = model.eval(pars, x=fitX)

    fit = np.row_stack((fit, fitY))

    # Plot your data and the fit
    plt.figure(figsize=(5, 3))
    plt.plot(field_used, signal, 'ko', label='Experimental', markersize=3)
    plt.plot(fitX, fitY, 'r-', label='Fit', linewidth=3)
    plt.xlabel('Field (Oe)')
    plt.ylabel('dP/dH (a.u.)')
    plt.title(f'FMR Spectrum Fitting')
    plt.legend()
    plt.grid(True)

    # Append parameter values and uncertainties to the DataFrame
    parameter_values_df = parameter_values_df.append({
        'Frequency (GHz)': freq_used[i],
        'K1': fit_result.params['pA_K1'].value,
        'K2': fit_result.params['pA_K2'].value,
        'H_FMR': fit_result.params['pA_H_FMR'].value,
        'DH': fit_result.params['pA_DH'].value,
        'Intercept': fit_result.params['bg_intercept'].value,
        'Slope': fit_result.params['bg_slope'].value,
        'K1_Uncertainty': fit_result.params['pA_K1'].stderr,  # Add uncertainties
        'K2_Uncertainty': fit_result.params['pA_K2'].stderr,
        'H_FMR_Uncertainty': fit_result.params['pA_H_FMR'].stderr,
        'DH_Uncertainty': fit_result.params['pA_DH'].stderr,
        'Intercept_Uncertainty': fit_result.params['bg_intercept'].stderr,
        'Slope_Uncertainty': fit_result.params['bg_slope'].stderr,
    }, ignore_index=True)

    ################## CHANGE DIRECTORY HERE #####################

    output_directory = r"C:\Users\physlab\Desktop\August_data\NiFe_2Mode_R\Parameters"
    
    plot_filename = os.path.join(output_directory, f'Fitted_plot_%.2f_GHz.png' %freq[i])
    plt.savefig(plot_filename)

# Save the updated DataFrame to the existing CSV file
csv_file_path = output_directory + '\\parameter_values.csv'
parameter_values_df.to_csv(csv_file_path, index=False)

fit = fit[1:][:]
```

### You can then use the fitted data to view the density plots.

## Step 6: Kittel Fitting

* Using the fitted data, call the relevant columns in the Pandas dataframe

```python
from numpy import pi, sqrt

data = pd.read_csv(csv_file_path)
freq = data.iloc[:, 0]
H_FMR = data.iloc[:, 3]
line_width = data.iloc[:, 4].values

#Uncertainties in parameters
H_FMR_stderr = data.iloc[:, 9]
line_width_stderr = data.iloc[:, 10]
```

### Set Kittel parameters
```python
H = np.linspace(0, 600, len(H_FMR)) # field range

gamma = 17.60E6/1E9 # gyromagnetic ratio
Ms = 590 # magnetic saturation
ku = 0.52  # unaxial anistropy constant
T = 20 *1E7 # thickness
Hu = 2500 #2*ku/(Ms*T) # perpendiculer anisotropy field
# ku = Hu/2 * Ms
Meff = Ms - (Hu/4*pi) # effective magnetization 
Hk = 4   # uniaxial anistropy field
Hr = 0   # rotatable anistropy field
```

### Finally, plot the Kittel results

```python
# Kittel Equation
def kittel_equation():
    return gamma/(2*pi) * sqrt((4*pi*Ms + Hu + H + Hk + Hr) * (H + Hk + Hr))

kittel_results = kittel_equation()

# Plotting
plt.figure(figsize=(6, 4))
plt.plot(H_FMR, freq, 'ko', markersize=3, label='Experimental')
plt.plot(H, kittel_results, 'g-', linewidth=1, label='Kittel Fit')
plt.xlabel('H_FMR')
plt.ylabel('Frequency (GHz)')
plt.legend()
plt.grid()
plt.title('Kittel Fitting')
plt.show()

plt.figure(figsize=(6,4))
smoothed_plot(field_used, freq_used, fit, filter_type='savgol', second_filter_type='gaussian')
plt.plot(H, kittel_results, 'g', linewidth=2, label='Kittel Fit')
plt.title('Kittel Fitting On Density Plot')
plt.show()

plt.figure(figsize=(6,4))
smoothed_plot(field_used, freq_used, integrated, filter_type='fft', second_filter_type='gaussian')
plt.plot(H, kittel_results, 'g', linewidth=2, label='Kittel Fit')
plt.title('Kittel Fitting On Integrated Density Plot')
plt.show()

print(f'Gyromagnetic Ratio (γ): {gamma:.2e} Hz/Oe')
print(f'Saturation Magnetization (Ms): {Ms:.2e} emu/cm^3')
print(f'Effective Magnetization (Meff): {Meff:.2e} emu/cm^3')
print(f'Uniaxial Anistropy (ku): {ku} erg/cm^2')
```

## Step 7: Linear Fitting

* Use linear regression to plot Line Width against Frequency.
* Line Width > 3 standard deviations are considered outliers and not used in linear fitting.
* Important statistical data is printed at the end.

```python
freq = data.iloc[:, 0].values

# Linear Regression
slope, intercept, r_value, p_value, std_err = linregress(freq, line_width)

# Creating a linear fit line using the slope and intercept
fit_line = slope * freq + intercept

outlier_indices = np.where(np.abs(line_width - fit_line) > 3 * np.std(fit_line))[0]

freq_without_outliers = np.delete(freq, outlier_indices)
line_width_without_outliers = np.delete(line_width, outlier_indices)

slope, intercept, r_value, p_value, std_err = linregress(freq_without_outliers, line_width_without_outliers)
fit_line = slope * freq + intercept

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(freq, line_width, 'ko', markersize=3, label='Experimental')
plt.plot(freq[outlier_indices], line_width[outlier_indices], 'ro', 
         markersize=10, label='Outliers', markerfacecolor='none', markeredgewidth=1)
plt.errorbar(freq, line_width, line_width_stderr, fmt='o', color='black', markersize=3, ecolor='red')
plt.plot(freq, fit_line, color='blue', alpha=0.5)
plt.title('Linear Fitting')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Line Width (Oe)')
plt.legend()
plt.grid()
plt.show()

# Linear Regression Results
print('Results:\n')
print(f'Slope: {slope:.4f}')
print(f'Intercept: {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')
print(f'Standard Error: {std_err:.4f}')

alpha = (gamma*slope)/(2*pi)
print(f'Gilbert Damping Parameter (alpha): {alpha: .4f}')
```
## Conclusion 
This tutorial guides you through the entire process of FMR Model Fitting using the provided code. Make sure to adapt the directory paths and parameters according to your specific dataset and requirements.